<a href="https://colab.research.google.com/github/hahajjjun/MLCompetition_Toy_Projects/blob/main/hard_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting.py

In [21]:
import pandas as pd
import random

def Normalize(arr):

  sum = 0
  for entry in arr:
    sum += entry
  
  for i in range(len(arr)):
    arr[i] = arr[i]/sum
  return arr

# Init.py

In [27]:
#Hyperparameters
B,C,M = 3,3,8000
x = 20 #Lower Bound of depth
y = 30 #Upper Bound of depth
zero_probability = 0.4 #Probability of creating [0,0]

blocks = ["block"+str(i) for i in range(1,B+1)]
clones = ["clone"+str(i) for i in range(1,C+1)]

observation = dict()
for block in blocks:
  observation[block] = []

params = dict()
last_block = 'block' + str(B)
for block in blocks:
  params[block] = []
  for clone in clones:
    params[block].append(random.random())

#Normalization
for block in blocks:
    params[block] = Normalize(params[block])

print("----Hyperparameters----")
print(f"Blocks : {B} \nClones : {C} \nMutations : {M} \nDepth Range : ({x},{y}) \nZero Probability : {zero_probability} \nParams : {params}")

----Hyperparameters----
Blocks : 3 
Clones : 3 
Mutations : 8000 
Depth Range : (20,30) 
Zero Probability : 0.4 
Params : {'block1': [0.6340804411839511, 0.18360861455189945, 0.18231094426414945], 'block2': [0.06105310029533281, 0.37397995964760455, 0.5649669400570626], 'block3': [0.5321535854763867, 0.20662956152247133, 0.26121685300114195]}


# Input.py

In [23]:
answer_membership = []
mutation_cnt = 0

while mutation_cnt < M:
  membership = random.choice(clones)
  zero_cnt = 0
  mutation_input = dict()

  for block in blocks:
    #Depth & Alt Allocation
    if random.choice(range(10)) < zero_probability*10:
      input = [0,0]
      zero_cnt += 1
    else:
      depth = random.randrange(x,y)
      alt = int(depth*params[block][clones.index(membership)]/2)
      input = [depth, alt]

    mutation_input[block] = input
    
  #Check if there is mutation
  if zero_cnt != 3:
    for block in blocks:
      observation[block].append(mutation_input[block])
    answer_membership.append(membership)
    mutation_cnt += 1

observation_df = pd.DataFrame(observation)
observation_df['answer'] = answer_membership

# Clustering.py

In [28]:
new_df = pd.DataFrame()
i = 0
for column in observation_df.columns:
  i += 1
  if column != 'answer':
    freq_list = []
    for items in observation_df[column]:
      depth, alt = items[0], items[1]
      if depth != 0:
        normal = depth - alt
        freq_list.append(alt/depth)
      else:
        freq_list.append(0)
    new_df[i] = freq_list
new_df

,1,2,3
0,0.142857,0.379310,0.200000
1,0.142857,0.035714,0.068966
2,0.000000,0.038462,0.041667
3,0.000000,0.038462,0.000000
4,0.130435,0.034483,0.037037
...,...,...,...
7995,0.000000,0.000000,0.208333
7996,0.136364,0.375000,0.178571
7997,0.000000,0.038462,0.206897
7998,0.136364,0.040000,0.000000


In [29]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=C, random_state=0).fit(new_df)
gmm_cluster_labels = gmm.predict(new_df)
print(gmm_cluster_labels)

[1 0 0 ... 0 0 1]


In [26]:
table = pd.DataFrame(columns = ["predict", "answer"])
table["predict"] = gmm_cluster_labels
table["answer"] = observation_df['answer']
link = {"clone1":0, "clone2":0, "clone3":0}
for clone in clones:
  zeros = sum(table[table["answer"] == clone]['predict'] == 0)
  ones = sum(table[table["answer"] == clone]['predict'] == 1)
  twos = sum(table[table["answer"] == clone]['predict'] ==2)
  cnts = [zeros, ones, twos]
  link[clone] = cnts.index(max(cnts))
table.answer = table.answer.map(link)
print('Accruacy:', sum(table.predict==table.answer)/len(table))

Accruacy: 0.797
